##Librerías varias que utilizaremos.

In [0]:
import requests, six
import lxml.html as lh
import pandas as pd

##Creamos los dataset con los datos del misal

In [0]:
url = 'https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/'

In [3]:
#Para realizar una solicitud HTTP
page = requests.get(url)

#Almacenar los contenidos de la web.
doc = lh.fromstring(page.content)

#Los datos que esten entre <tr>...</tr>
tr_elements = doc.xpath('//tr')

#Aquí checamos si las filas tienen el mismo largo
[len(T) for T in tr_elements[2:19]]

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [4]:
#Creamos una lista vacía.
col=[('Región',[]),('Casos totales acumulados',[]),('Casos nuevos totales',[]),('Casos nuevos con síntomas',[]),('Casos nuevos sin síntomas*',[]),('Fallecidos',[]),('% Total',[])]
i=0

#Guardamos el encabezado y una lista vacia (primera tabla).
'''for t in tr_elements[1]:
    i+=1
    name=t.text_content()
    name=name.replace('\n',' ')
    print('%d:"%s"'%(i,name))
    col.append((name,[]))'''

#print(col)
casosR = []
i=0
#Guardamos el encabezado y una lista vacía (segunda tabla).
for u in tr_elements[19]:
    i+=1
    name_2=u.text_content()
    print('%d:"%s"'%(i,name_2))
    if i > 1:
      name_2 = name_2.replace('.','')
      name_2 = int(name_2)
      casosR[0][1].append(name_2)
      break
    casosR.append((name_2,[]))
print(casosR)
#El segundo for lo usamos para obtener los datos de la segunda tabla, ya que el largo de la fila de la segunda tabla es distinto.

1:"Casos recuperados a nivel nacional"
2:"8.580"
[('Casos recuperados a nivel nacional', [8580])]


In [0]:
for j in range(2,len(tr_elements)):

    T=tr_elements[j]
    
    #Esto para que no guarde filas que tengas menos o más de 5 columnas.
    if len(T)!=7:
        break
  
    i=0
    
    #Recorremos cada elemento que tenga la fila.
    for t in T.iterchildren():
        data=t.text_content() 
        #Compruebe si la fila está vacía.
        if i>0:
        #Convierte cualquier valor numérico a enteros.
          data = data.replace('.',',')
          try:
            data=int(data)
          except:
            pass
        #Agrega el valor a la lista vacía en la columna i.
        col[i][1].append(data)
        i+=1

In [8]:
[len(C) for (title,C) in col]

[17, 17, 17, 17, 17, 17, 17]

##Creamos el primer dataset con los valores de la página.

In [9]:
Dict={title:column for (title,column) in col}
import pandas as pd
df=pd.DataFrame(Dict)
df

,Región,Casos totales acumulados,Casos nuevos totales,Casos nuevos con síntomas,Casos nuevos sin síntomas*,Fallecidos,% Total
0,Arica y Parinacota,279,7,2,5,4,"1,74%"
1,Tarapacá,198,15,15,0,1,"1,24%"
2,Antofagasta,546,32,27,5,5,"3,41%"
3,Atacama,43,1,1,0,0,"0,27%"
4,Coquimbo,76,0,0,0,0,"0,47%"
5,Valparaíso,559,33,28,5,12,"3,49%"
6,Metropolitana,"9,625",736,650,86,110,"60,07%"
7,O’Higgins,108,4,3,1,1,"0,67%"
8,Maule,393,1,0,1,13,"2,45%"
9,Ñuble,768,7,7,0,14,"4,79%"


##Creamos el segundo dataset con los datos totales de cada columna

Esto es para tener ordenada la tabla y para que luego sea más fácil la manipulación de los datos.

In [0]:
Dic_2 = {title:column for (title,column) in casosR}

In [13]:
import datetime
from datetime import date

fw = pd.DataFrame(Dic_2)
R=df.iloc[16:,5:6].values #Fallecidos
X=df.iloc[16:,2:3].values #Casos nuevos
Y=df.iloc[16:,1:2].values #Casos Totales
#Y = int(Y) 
dia = str(date.today()) #Fecha
#print(R)
#print(X)
#print(Y)

fw = fw.assign(Casos_fallecidos=R)
fw = fw.assign(Casos_totales_dia=Y)
fw = fw.assign(Casos_totales_nuevos=X)
fw = fw.assign(Fecha=dia)
fw

,Casos recuperados a nivel nacional,Casos_fallecidos,Casos_totales_dia,Casos_totales_nuevos,Fecha
0,8580,227,16023,888,2020-04-30


##Este es el dataset que ocupamos para hacer el gif

In [14]:
import os
import csv 
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/COVID-19 Chile (Coronavirus)/Database (CSV)/csv') #En donde vamos a guardar los archivos CSV. 

twq = pd.read_csv('Casos_Totales_Region_.csv', sep=',')
twq

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


,Arica y Parinacota,Tarapacá,Antofagasta,Atacama,Coquimbo,Valparaíso,Metropolitana,O’Higgins,Maule,Ñuble,Biobío,Araucanía,Los Ríos,Los Lagos,Aysén,Magallanes,Fecha
0,0,0,0,0,0,0,7,0,4,0,1,0,0,1,0,0,2020-03-09
1,0,0,0,0,0,0,10,0,5,0,1,0,0,1,0,0,2020-03-10
2,0,0,0,0,0,0,14,0,7,0,1,0,0,1,0,0,2020-03-11
3,0,0,0,0,0,0,23,0,7,1,1,0,0,1,0,0,2020-03-12
4,0,0,0,0,0,0,29,0,8,2,3,0,0,1,0,0,2020-03-13
5,0,0,2,1,0,0,40,0,9,4,3,0,0,1,1,0,2020-03-14
6,0,0,2,1,0,0,54,0,9,4,3,0,0,1,1,0,2020-03-15
7,0,0,2,1,0,1,123,0,9,12,4,1,1,1,1,0,2020-03-16
8,0,0,2,1,0,1,152,0,9,26,4,1,1,1,1,2,2020-03-17
9,0,0,2,1,0,1,174,0,11,26,7,4,1,8,1,2,2020-03-18


In [0]:
#actualizamos el dataset
data = []

for t in df.iloc[:-1,1:2].values: #Casos Totales
  data.append(int(t))
data.append(str(date.today()))
twq.loc[len(twq)] = data

In [16]:
#Visualización del dataset
twq

,Arica y Parinacota,Tarapacá,Antofagasta,Atacama,Coquimbo,Valparaíso,Metropolitana,O’Higgins,Maule,Ñuble,Biobío,Araucanía,Los Ríos,Los Lagos,Aysén,Magallanes,Fecha
0,0,0,0,0,0,0,7,0,4,0,1,0,0,1,0,0,2020-03-09
1,0,0,0,0,0,0,10,0,5,0,1,0,0,1,0,0,2020-03-10
2,0,0,0,0,0,0,14,0,7,0,1,0,0,1,0,0,2020-03-11
3,0,0,0,0,0,0,23,0,7,1,1,0,0,1,0,0,2020-03-12
4,0,0,0,0,0,0,29,0,8,2,3,0,0,1,0,0,2020-03-13
5,0,0,2,1,0,0,40,0,9,4,3,0,0,1,1,0,2020-03-14
6,0,0,2,1,0,0,54,0,9,4,3,0,0,1,1,0,2020-03-15
7,0,0,2,1,0,1,123,0,9,12,4,1,1,1,1,0,2020-03-16
8,0,0,2,1,0,1,152,0,9,26,4,1,1,1,1,2,2020-03-17
9,0,0,2,1,0,1,174,0,11,26,7,4,1,8,1,2,2020-03-18


##Por último creamos y actualizamos los archivos CSV

In [17]:
#print(date.today())
#fw.to_csv('Resumen_COVID-19_Chile_'+dia+'.csv', header=True, index=False) #Para crear todos los días un csv distinto con los datos de la segunda tabla.
twq.to_csv('Casos_Totales_Region_.csv', header=True, index=False) #Para crear todos los dias un csv distinto. #actualizamos el dataset para el gif

#Aqui nos evitamos crear un csv para cada dia y tan solo actualizamos todos los dias el que ya existe (segunda tabla).
with open(r'Resumen_COVID-19_Chile_.csv', 'a') as f:
    writer = csv.writer(f)
    for row in fw.iloc[:,:].values:
      writer.writerow(row)

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/COVID-19 Chile (Coronavirus)/Database (CSV)/csv/Dataset diarios') #En donde vamos a guardar los archivos CSV. 
df.drop(df.index[[16]],inplace=True)#Esto es para borrar los datos totales (ultima fila) de la primera tabla, porque recuerden que los vamos a guardar en la segunda.
df.to_csv('Casos_COVID-19_Chile_'+dia+'.csv', header=True, index=False) #Creamos el primer archivo con la primera tabla.

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/COVID-19 Chile (Coronavirus)/Database (CSV)/xlsx') #En donde vamos a guardar los archivos xlsx. 
twq.to_excel('Casos_Totales_Region_.xlsx', header=True, index=False) #Para crear todos los dias un xlsx distinto.

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/COVID-19 Chile (Coronavirus)/Database (CSV)/xlsx/Datasets diarios') #En donde vamos a guardar los archivos xlsx.
df.to_excel('Casos_COVID-19_Chile_'+dia+'.xlsx', header=True, index=False) #Creamos el primer archivo con la primera tabla.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
